In [1]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
from dgl.nn.pytorch.conv import SGConv
from time import perf_counter

d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lr = 0.2
bias = False
epochs = 50
weight_decay = 5e-4
loss_fn = torch.nn.CrossEntropyLoss()

In [3]:
data = PubmedGraphDataset()
g = data[0]
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\dgl\data\utils.py:288: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [4]:
model = SGConv(in_feats, n_classes, k=2, cached=True, bias=bias)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [5]:
def test(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [6]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    t = perf_counter()
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')
    print(f'Time: {perf_counter()-t:.2f}s')

In [7]:
train(model, g, features, labels, train_mask, val_mask, test_mask)
test(model, g, features, labels, test_mask)

Epoch 01, Loss: 1.0993, Train: 0.9000, Val: 0.7080, Test: 0.6970
Epoch 02, Loss: 1.0162, Train: 0.9167, Val: 0.7200, Test: 0.7140
Epoch 03, Loss: 0.9454, Train: 0.9167, Val: 0.7360, Test: 0.7260
Epoch 04, Loss: 0.8881, Train: 0.9500, Val: 0.7340, Test: 0.7320
Epoch 05, Loss: 0.8437, Train: 0.9500, Val: 0.7440, Test: 0.7370
Epoch 06, Loss: 0.8107, Train: 0.9500, Val: 0.7520, Test: 0.7380
Epoch 07, Loss: 0.7871, Train: 0.9500, Val: 0.7560, Test: 0.7410
Epoch 08, Loss: 0.7711, Train: 0.9667, Val: 0.7560, Test: 0.7500
Epoch 09, Loss: 0.7610, Train: 0.9667, Val: 0.7560, Test: 0.7530
Epoch 10, Loss: 0.7554, Train: 0.9667, Val: 0.7560, Test: 0.7550
Epoch 11, Loss: 0.7530, Train: 0.9667, Val: 0.7580, Test: 0.7580
Epoch 12, Loss: 0.7528, Train: 0.9667, Val: 0.7680, Test: 0.7600
Epoch 13, Loss: 0.7541, Train: 0.9667, Val: 0.7720, Test: 0.7610
Epoch 14, Loss: 0.7561, Train: 0.9667, Val: 0.7720, Test: 0.7570
Epoch 15, Loss: 0.7585, Train: 0.9667, Val: 0.7720, Test: 0.7550
Epoch 16, Loss: 0.7609, T

0.759